In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder

In [3]:
# 데이터 불러오기
sample_path = '../input/pubg-finish-placement-prediction/sample_submission_V2.csv' 
train_path = '../input/pubg-finish-placement-prediction/train_V2.csv'
test_path = '../input/pubg-finish-placement-prediction/test_V2.csv'
sample = pd.read_csv(sample_path)
train= pd.read_csv(train_path)
test=pd.read_csv(test_path)

In [4]:
#test_o는 불러온 데이터의 원본을 복사, df_o는 불러온 train데이터의 원본을 복사하고 타겟벨류의 결측치 제거, df는 train에 활용하기 위해 복사
test_o=test.copy()
df_o= pd.read_csv(train_path)
df_o= df_o.dropna()
df=df_o.copy()

In [5]:
#데이터 타입 최적화-> 메모리 최적화
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
df = reduce_mem_usage(df)
test = reduce_mem_usage(test)

In [6]:
#함수로 만들자
def feature_engineering(df):
    #수치형 데이터
    df_col= [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills','matchType',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']
    # df_t는 데이터를 그룹아이디 기준으로 평균을 내기 위해 저장할 데이터프레임
    df_t=df.copy()
    #라벨인코딩
    le = LabelEncoder()
    df_t['matchType']= le.fit_transform(df_t['matchType'])
    df_t['matchType']
    # 그룹아이디를 기준으로 수치형 컬럼들 평균화
    df_t= df_t.groupby(['groupId'])[df_col].agg('mean').reset_index()


    
    return df_t
    

In [7]:
#df에는 앞에서 만든 함수를 통해 피처 엔지니어링 수치형 데이터를 평균을 냄
df = feature_engineering(df)
test = feature_engineering(test)

In [8]:
#원본 train 데이터의 순서와 로우를 잃지 않기 위해 그룹아이디와 그에 맞는 타겟벨류를 가지고옴
train_t=df_o.copy()
train_t=train_t[['groupId','winPlacePerc']]
# 데이터 합치기- 앞에서 데이터를 피처 엔지니어링한 평균을 원본 train 데이터의 순서에 맞게 넣기 위해 merge를 사용 how='left'는 
#두 데이터프레임중 기준을 잡을 데이터프레임을 정함, 원본train의 순서와 로우수를 맞춰줌
train_t=pd.merge(train_t,df,how='left',on='groupId')
# 모델 학습을 위해 두가지 변수를 제거 killplace는 큰 비중을 차지하여 공부를 위해 제거 groupId는 학습을 위해 수치형 데이터만
#남게 하기 위해 제거 
train_t=train_t.drop(columns=['killPlace','groupId'])
train_t
    

In [9]:
#원본 데이터의 순서와 로우 수를 위해 그룹아이디를 가지고옴
test_t=test_o.copy()
test_t=test_t[['groupId']]
#데이터 합치기- 위와 동일
test_t=pd.merge(test_t,test,how='left',on='groupId')
test_t=test_t.drop(columns=['killPlace','groupId'])
test_t

In [10]:
#모델링을 위해 winplacePerc를 제거하고, y로 값을 따로 만듬
x=train_t.drop(columns='winPlacePerc')
y=train_t[['winPlacePerc']]

In [11]:
#스케일링, test_t도 스케일링을 함께 함
scaler = StandardScaler()
test_t=scaler.fit_transform(test_t)
x=scaler.fit_transform(x)
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=0)
x_train.shape,x_val.shape,y_train.shape,y_val.shape

In [12]:
# lightgbm 활용-> 학습용
import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
d_val = lgb.Dataset(x_val,label=y_val)
params = {"objective" : "regression", "metric" : "mae", 'n_estimators':1000, 'early_stopping_rounds':200,
              "num_leaves" : 31, "learning_rate" : 0.05, "bagging_fraction" : 0.7,
               "bagging_seed" : 0, "num_threads" : 4,"colsample_bytree" : 0.7
             }

clf = lgb.train(params, d_train, valid_sets=[d_train, d_val],early_stopping_rounds=200, verbose_eval=1000)
# pred = clf.predict(x_test, num_iteration=clf.best_iteration)
#mae=mean_absolute_error(y_test,pred)
print(clf)



In [13]:
# 학습한 모델 clf로 테스트파일을 예측함 pred는 테스트 파일의 y벨류가 들어있음
pred = clf.predict(test_t, num_iteration=clf.best_iteration)

In [14]:
# 제출을 위해 파일 변환
df1 = pd.read_csv('../input/pubg-finish-placement-prediction/sample_submission_V2.csv')
df1['winPlacePerc']=pred
df1.to_csv('submission.csv',index = False)